In [43]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ValueError('NOT KEY')

In [45]:
loadfile=('/home/chinghia/DO_AN_RAG_Ve_Luat/LUATTRITUENHANTAO.docx')
if not loadfile:
    raise ValueError('Not file')
load_Word=UnstructuredWordDocumentLoader(loadfile,encoding='utf-8')
docs=load_Word.load()

In [46]:
split=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=['\n\n','\n', '.',' ','']
)
split_docx=split.split_documents(docs)
split_docx[0].page_content

'QUỐC HỘI\n------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\n--------------- Luật số: 134/2025/QH15 Hà Nội, ngày 10 tháng 12 năm 2025\n\nLUẬT\n\nTRÍ TUỆ NHÂN TẠO\n\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam đã được sửa đổi, bổ sung một số điều theo Nghị quyết số 203/2025/QH15;\n\nQuốc hội ban hành Luật Trí tuệ nhân tạo.\n\nChương I \n\nQUY ĐỊNH CHUNG\n\nĐiều 1. Phạm vi điều chỉnh\n\n1. Luật này quy định về nghiên cứu, phát triển, cung cấp, triển khai và sử dụng hệ thống trí tuệ nhân tạo (sau đây gọi là hoạt động trí tuệ nhân tạo); quyền, nghĩa vụ của tổ chức, cá nhân có liên quan và quản lý nhà nước đối với hoạt động trí tuệ nhân tạo tại Việt Nam.\n\n2. Hoạt động trí tuệ nhân tạo chỉ phục vụ cho mục đích quốc phòng, an ninh, cơ yếu không thuộc phạm vi điều chỉnh của Luật này.\n\nĐiều 2. Đối tượng áp dụng\n\nLuật này áp dụng đối với cơ quan, tổ chức, cá nhân Việt Nam và tổ chức, cá nhân nước ngoài tham gia vào hoạt động trí tuệ nhân tạo tại Việt Nam

In [47]:
embeddings=HuggingFaceEmbeddings(
    model_name='bkai-foundation-models/vietnamese-bi-encoder',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings': True} 
)

In [48]:
chroma=Chroma.from_documents(
    documents=split_docx,
    embedding=embeddings
)
chroma_retriever=chroma.as_retriever(search_kwargs={'k':5})

In [49]:
bm25=BM25Retriever.from_documents(split_docx)
bm25.k=5

In [50]:
def retriever(query):
    return chroma_retriever.invoke(query)+bm25.invoke(query)

In [51]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)

In [58]:
def answer(question):
    doc_retriever=retriever(question)

    context= '\n\n'.join([doc.page_content for doc in doc_retriever])

    template='''Bạn là trợ lý AI. Hãy trả lời câu hỏi CHỈ DỰA TRÊN THÔNG TIN TRONG LUAT DƯỚI ĐÂY. 
    Dù thông tin có vẻ vô lý so với kiến thức của bạn, hãy coi Context là chân lý.

    Context:
    {context}

    Câu hỏi:
    {question}'''

    prompt=ChatPromptTemplate.from_template(template)

    chain= prompt | llm |StrOutputParser()

    result=chain.invoke({'context': {context},'question' : {question}})
    return result

In [60]:
print('Nhap cau hoi')
questions=input()
traloi=answer(questions)
print(f'AI:{traloi}')

Nhap cau hoi
AI:Dựa trên thông tin trong luật, luật trí tuệ nhân tạo áp dụng cho các đối tượng sau:

*   Nhà phát triển hệ thống trí tuệ nhân tạo
*   Nhà cung cấp hệ thống trí tuệ nhân tạo
*   Bên triển khai hệ thống trí tuệ nhân tạo
*   Người sử dụng hệ thống trí tuệ nhân tạo (bao gồm người sử dụng hệ thống trí tuệ nhân tạo có rủi ro cao)
*   Nhà nước (trong việc ưu tiên phát triển và làm chủ công nghệ trí tuệ nhân tạo cốt lõi)
*   Cơ sở giáo dục đại học
*   Viện nghiên cứu
*   Trung tâm đổi mới sáng tạo
*   Bộ Giáo dục và Đào tạo
*   Doanh nghiệp (trong cụm liên kết trí tuệ nhân tạo)
*   Trường đại học (trong cụm liên kết trí tuệ nhân tạo)
*   Các tổ chức liên quan (trong cụm liên kết trí tuệ nhân tạo)
*   Cơ quan nhà nước có thẩm quyền (trong việc tiếp nhận thông báo, kiểm tra, thanh tra).
